In [1]:
# Imports

import math
import numpy as np
from matplotlib import pyplot as plt
from numpy import random as rnd
import warnings,datetime,os,calendar,csv,time

import tensorflow as tf
import pandas as pd
import seaborn as sns

from keras.models import Model,Sequential
from keras.layers import Dense,LSTM,Conv2D,Dropout,BatchNormalization,Input,Concatenate,Add,Activation,MaxPooling2D,AveragePooling2D
import keras.backend as K

from sklearn import preprocessing as pp

from sklearn.cluster import KMeans,MeanShift
from sklearn.dummy import DummyClassifier,DummyRegressor
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import AdaBoostClassifier,AdaBoostRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.linear_model import BayesianRidge,Lasso,LinearRegression,SGDClassifier,SGDRegressor
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier,RadiusNeighborsRegressor,NearestNeighbors
from sklearn.manifold import Isomap,TSNE
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.svm import LinearSVC,LinearSVR
from sklearn.neural_network import BernoulliRBM,MLPClassifier,MLPRegressor
from sklearn.decomposition import FactorAnalysis,KernelPCA,PCA,MiniBatchSparsePCA,FastICA
from sklearn.preprocessing import CategoricalEncoder,KBinsDiscretizer,LabelEncoder,MinMaxScaler,OneHotEncoder,StandardScaler

from xgboost import XGBClassifier,XGBRegressor

import gym
import pygame
from pygame.locals import *

import pickle,h5py,json

import pandas_datareader as pdr
import urllib
from bs4 import BeautifulSoup
from scipy.signal import resample,correlate
from scipy import fftpack
from lmfit import Model as lmmodel

import iexfinance as iex
from iexfinance.stocks import Stock,get_historical_data

warnings.filterwarnings('ignore')
sns.set()
plt.xkcd()

C:\Users\Jan\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class IEX_Company:
    
    def __init__(self,ID):
        self.ID = ID
        self.token = None
        self.base_initialized = False
        self.hist_initialized = False
    
    def set_token(self,token):
        if self.token is not None: print('Refreshing authentication token...')
        self.token = token
        return
        
    def fetch_base_data(self):
        if self.token is None: 
            print('Please set authentication token first!')
            return
        if self.base_initialized: print('Refreshing base data...')
        
        stock = Stock(self.ID,token=self.token,output_format='pandas')
        
        self.peers = stock.get_peers(token=self.token)[self.ID]
        self.df_b = stock.get_balance_sheet(token=self.token)
        self.df_i = stock.get_company(token=self.token)
        self.df_c = stock.get_cash_flow(token=self.token)
        
        if not self.base_initialized: self.base_initialized=True
        return
    
    def fetch_historical_data(self,start_date,end_date):
        if self.token is None: 
            print('Please set authentication token first!')
            return
        if self.hist_initialized: print('Refreshing base data...')
        
        self.df_h = get_historical_data(self.ID,start=start_date,end=end_date,token=self.token,output_format='pandas')
        
        if not self.hist_initialized: self.hist_initialized=True
        return
    
    def prepare_timeline(self,lookback,day_freq=1):
        if not self.hist_initialized:
            print('Please initialize historical data first!')
            return
        data = df_h.transpose().values
        r_data = data[:,::day_freq]

        num_features = r_data.shape[0]

        DATA = []
        TARGET = []

        for k in range(r_data.shape[1]-lookback-1):
            D = r_data[:,k:k+lookback]
            T = r_data[:,k+lookback]
            DATA.append(D)
            TARGET.append(T)
        DATA = np.swapaxes(np.stack(DATA,axis=0),1,2)
        TARGET = np.stack(TARGET,axis=0)
        
        self.DATA = DATA
        self.TARGET = TARGET
        return DATA,TARGET